# **RAG**
Retrieval Augmented Generation

## **1.환경준비**

### (1) 라이브러리 Import

In [1]:
import pandas as pd
import numpy as np
import os
import openai

from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, Document
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### (2) OpenAI API Key 확인
* 환경변수로 등록된 Key 확인하기

In [2]:
# 환경변수에서 키 불러오기
api_key = os.getenv('OPENAI_API_KEY')
print(api_key)

sk-proj-VJtHk7f7d42GRBoawHc1T3BlbkFJClYyPyOncU8oOuT1kdVF


* 만약 환경변수 키 설정이 잘 안된다면 아래 코드셀의 주석을 해제하고, 자신의 api key를 입력하고 실행
    * 아래 코드는 키 지정을 **임시**로 수행함.
    * 파이썬 파일(.ipynb, .py)안에서 매번 수행해야 함.

In [ ]:
# os.environ['OPENAI_API_KEY'] = '여러분의 OpenAI API키'
# openai.api_key = os.getenv('OPENAI_API_KEY')

## **2.Vector DB**

### (1) Chroma DB 구성

* DB 경로 지정
    * 없으면, 새로 폴더를 만들며 DB 생성
    * 있으면, 기존 DB 연결

In [3]:
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

database = Chroma(persist_directory="./database",  # 경로 지정(현 위치에서 db 폴더 생성)
                    embedding_function = embeddings  # 임베딩 벡터로 만들 모델 지정
)

* DB 삭제는, 파이썬을 닫고 폴더를 삭제하면 됩니다.

## **3.데이터 벡터화**

### (1) DF to Vector DB

* 샘플데이터 : 오픈소스 생성형 AI에서 주의해야 할 10가지 사항
    * 기사를 csv로 만든 것입니다.

원문보기: https://www.ciokorea.com/news/337152#csidxc7d1d11066fad86a15937e4c3b29c6d



In [4]:
data = pd.read_csv('./aivleschool_qa.csv', encoding='utf-8')
data

,구분,QA
0,모집/선발,최종 학력 또는 전공과 관계없이 지원할 수 있나요?\r\nKT 에이블스쿨은 정규 4...
1,모집/선발,35세 이상은 지원할 수 없나요?\r\n본 교육 과정은 34세 이하를 대상으로 하는...
2,모집/선발,미취업자의 기준이 뭔가요?\r\n미취업자의 기준은 아래와 같습니다.\r\n1) 기간...
3,모집/선발,"직장인도 지원할 수 있나요?\r\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육..."
4,모집/선발,아르바이트를 하고 있는데 지원할 수 있나요?\r\n고용보험에 가입이 되어 있는 경우...
...,...,...
63,기타,AICE (기존 AIFB) 자격증이 있어야 유리한가요?\r\n네. AICE (기존 ...
64,기타,AICE 시험관련 문의는 어디에 해야하나요?\r\nhelp@aice.study 으로...
65,기타,국민취업지원제도를 신청하고 싶습니다.\r\n국민취업지원제도 신청과 관련 사항은 거주...
66,기타,국민취업지원제도를 안해도 교육 참여되나요?\r\n국민취업제도를 신청하지 않아도 KT...


In [5]:
# 데이터프레임의 텍스트 열(시리즈)을 리스트로 변환

# 리스트 내용을 각각 document로 변환
documents = [Document(page_content= row['QA'], metadata = {'구분': '기타'}) for _, row in data.iterrows()]

print(documents)

[Document(page_content='최종 학력 또는 전공과 관계없이 지원할 수 있나요?\r\nKT 에이블스쿨은 정규 4년제 대학 졸업자 및 졸업예정자를 대상으로 하는  교육입니다. 전공에 관계 없이 명시된 지원 자격에 부합한다면 모두 지원 가능합니다. 다만, AI개발자 Track은 기본적인 코딩역량이 필요합니다.', metadata={'구분': '기타'}), Document(page_content='35세 이상은 지원할 수 없나요?\r\n본 교육 과정은 34세 이하를 대상으로 하는 교육입니다. 단, 모집시점에 35세라도 해당연도 1월 1일 이후 생일자는 지원이 가능합니다. (예: 5기 지원 기준 1989년 1월 1일 이후 출생자)', metadata={'구분': '기타'}), Document(page_content='미취업자의 기준이 뭔가요?\r\n미취업자의 기준은 아래와 같습니다.\r\n1) 기간의 정함이 있는 근로인 경우,\r\n2) 고용보험에 미가입한 경우,\r\n3) 고용보험에 가입되어 있더라도 15시간/주 미만 근로인 경우\r\n단, 어떠한 경우에도 교육을 풀타임(09:00~18:00)으로 들을 수 있어야 교육 참여가 가능합니다.', metadata={'구분': '기타'}), Document(page_content='직장인도 지원할 수 있나요?\r\nKT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가능합니다. 향후 취업여부를 확인할 예정입니다.', metadata={'구분': '기타'}), Document(page_content='아르바이트를 하고 있는데 지원할 수 있나요?\r\n고용보험에 가입이 되어 있는 경우 15시간/주 미만 근로인 경우에만 미취업자로 간주하여 지원이 가능합니다.\r\n고용보험에 가입되어 있지 않는 경우에는 특수형태근로자/고용보험미가입근로자로 분류되어 근로시간과 훈련시간이 중복되지 않는다면 훈련 수강은 가능하나, 훈련장려금이 지급되지 않습니다.\r\n다만, 특수형태근로자의 경우 해촉증명서, 

In [6]:
# Insert
database.add_documents(documents)

['71d3cbcd-25b5-400a-8139-1bd8d4807126',
 '2dbefd64-847e-47c8-92b3-ab3ad7dec05b',
 'e4b19549-adb7-413e-8057-593b641d7bb2',
 '8df7970b-62b6-4e3b-b7ad-c708ec557aa0',
 'ef79590b-504d-42bd-9f96-e9bd6a5f5cb0',
 '75d721e2-44d4-47b6-8136-5443ae02eead',
 'b449d1bd-74e5-43f5-865c-7f2f145e5350',
 '154b5cf8-f921-446f-8328-a4b093fb5bba',
 '38a3dd56-3d43-40c8-b2f8-3124a0329909',
 '8c4a8a18-52cc-4072-8304-c694427d5182',
 '4b8127f7-67ca-4eb9-b87d-72f798aa4b67',
 'ebd03951-7e5b-421f-9d36-745b10e27864',
 'a30dfabf-231e-4eff-bbd7-9cd31bd13099',
 'd537a0bc-a854-4694-9a1b-0f1a7dc13b3e',
 '2ee2fcef-775a-4a24-bdd2-db72724dd4d3',
 '7937adb7-ccf4-47a0-9fab-030f403e160e',
 '3b5145f8-be28-4596-8eb7-9715b34f6542',
 'f83d55dc-fb37-4745-af82-aa68c4f5a043',
 '19d365ad-00e1-446b-ba59-eecc35012d4d',
 '0265e0cf-cd65-4e83-bdd9-dbb5b6feef7c',
 '8d52d626-2e21-4245-96f1-b1353314f965',
 '586a8cd7-6d55-4372-9ac9-da57533a326d',
 'c16a14da-86db-434a-8266-0f8980c672b1',
 '41fe7c8f-ef91-4b66-9e99-190bdbafcc3b',
 'e95c3398-7141-